In [92]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [93]:
crime = pd.read_csv('offenses_by_city_2013.csv')

In [94]:
crime.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,"1,861",0,0.000,nan,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,"2,577",3,0.000,nan,0,0,3,24,3,20,1,0.000
2,Akron Village,"2,846",3,0.000,nan,0,0,3,16,1,15,0,0.000
3,Albany,"97,956",791,8.000,nan,30,227,526,"4,090",705,"3,243",142,nan
4,Albion Village,"6,388",23,0.000,nan,3,4,16,223,53,165,5,nan


Prepare this data to model with multivariable regression (including data cleaning if necessary) according to this specification:

$$ Property crime = \alpha + Population + Population^2 + Murder + Robbery$$

The 'population' variable is already set for you, but you will need to create the last three features.  Robbery and Murder are currently continuous variables.  For this model, please use these variables to create  categorical features where values greater than 0 are coded 1, and values equal to 0 are coded 0.  You'll use this data and model in a later assignment- for now, just write the code you need to get the data ready.  Don't forget basic data cleaning procedures, either!  Do some graphing to see if there are any anomalous cases, and decide how you want to deal with them.

In [95]:
crime['Population'] = pd.to_numeric(crime['Population'].str.replace(',',''), errors='coerce').dropna()
crime['Robbery'] = pd.to_numeric(crime['Robbery'], errors='coerce').dropna()
crime = crime.rename(columns={'Murder and\nnonnegligent\nmanslaughter': 'Murder'})
crime['Murder'] = crime['Murder'].dropna().astype(int)

In [96]:
crime.columns

Index(['City', 'Population', 'Violent\ncrime', 'Murder',
       'Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2',
       'Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary',
       'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3'],
      dtype='object')

In [97]:
crime['Murder'].dtype

dtype('float64')

In [98]:
crime['robbery'] = np.where(crime['Robbery'] > 0, 1, 0)
crime['murder'] = np.where(crime['Murder'] > 0, 1, 0)
crime['pop'] = crime['Population'].dropna().astype(int)
crime['pop_s'] = np.sqrt(crime['pop'])

In [99]:
crime['pop_s'].describe()

count    348.000
mean     112.745
std      165.545
min       22.935
25%       54.800
50%       85.050
75%      135.739
max     2897.607
Name: pop_s, dtype: float64

In [112]:
features = crime

In [113]:
features.head()

,City,Population,Violent crime,Murder,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3,robbery,murder,pop,pop_s
0,Adams Village,1861.000,0,0.000,nan,0,0.000,0,12,2,10,0,0.000,0,0,1861.000,43.139
1,Addison Town and Village,2577.000,3,0.000,nan,0,0.000,3,24,3,20,1,0.000,0,0,2577.000,50.764
2,Akron Village,2846.000,3,0.000,nan,0,0.000,3,16,1,15,0,0.000,0,0,2846.000,53.348
3,Albany,97956.000,791,8.000,nan,30,227.000,526,"4,090",705,"3,243",142,nan,1,1,97956.000,312.979
4,Albion Village,6388.000,23,0.000,nan,3,4.000,16,223,53,165,5,nan,1,0,6388.000,79.925


In [119]:
features.drop(['City', 'Population', 'Violent\ncrime', 'Murder','Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2','Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary','Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3'], axis=1, inplace=True)

In [120]:
features.head()

,robbery,murder,pop,pop_s
0,0,0,1861.000,43.139
1,0,0,2577.000,50.764
2,0,0,2846.000,53.348
3,1,1,97956.000,312.979
4,1,0,6388.000,79.925
